# LangChainHub 各种链的集合

### 使用文件加载专用chains

In [8]:
from langchain.chains import load_chain 

from langchain import hub
prompt = hub.pull("wfh/pair-math-v1")


print(prompt)

input_variables=['answer_a', 'answer_b', 'question', 'ref_answer_1'] metadata={'lc_hub_owner': 'wfh', 'lc_hub_repo': 'pair-math-v1', 'lc_hub_commit_hash': 'b5c7443ca521be24424381278b2ca5c76dfff5748057c9bdee82724c6531ebbf'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Please act as an impartial judge and evaluate the quality of the responses provided by two AI assistants to the user question displayed below. Your evaluation should consider correctness and helpfulness. You will be given a reference answer, assistant A\'s answer, and assistant B\'s answer. Your job is to evaluate which assistant\'s answer is better. Begin your evaluation by comparing both assistants\' answers with the reference answer. Identify and correct any mistakes. Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision. Do not allow the length of the responses to influence your evaluation. Do not favor cer

# 自定义链
- 当通用链不满足的时候，可以自行构建实现特定的母的

In [18]:
from typing import List, Dict, Any,Optional
from langchain.callbacks.manager import (
    CallbackManagerForChainRun
)

from langchain.chains.base import Chain
from langchain.prompts.base import BasePromptTemplate
from langchain.base_language import BaseLanguageModel

In [32]:
class wiki_article_chain(Chain):
    """开发一个wiki文章生成器"""
    prompt: BasePromptTemplate
    llm: BaseLanguageModel
    out_key:str="text"

    @property
    def input_keys(self) ->List[str]:
        """将返回Prompt所需的所有键"""
        return self.prompt.input_variables
    
    @property
    def output_keys(self) ->List[str]:
        "将始终返回text键"
        return [self.out_key]
    
    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager:Optional[CallbackManagerForChainRun]=None,
    )-> Dict[str,Any]:
        """运行链"""
        prompt_value = self.prompt.format_prompt(**inputs)
        response = self.llm.generate_prompt(
            [prompt_value],callbacks=run_manager.get_child() if run_manager else None
        )
        if run_manager:
            run_manager.on_text("wike article is written")
        return  {self.out_key:response.generations[0][0].text}
    
    @property
    def _chain_type(self)-> str:
        """链类型"""
        return "wiki_article_chain"

In [35]:
from langchain.chat_models import  ChatOpenAI
from langchain.prompts import  PromptTemplate

chain = wiki_article_chain(
    prompt=PromptTemplate(
        template="写一篇关于{topic}的维基百科形式的文章",
        input_variables=["topic"]
    ),
    llm=ChatOpenAI(
        temperature=0
    ),
)

In [36]:
result = chain.run({"topic":"过好这一生"})
print(result)

过好这一生是每个人都追求的目标，它涉及到个人的幸福、成就和满足感。在人生的旅程中，每个人都希望能够过上有意义、充实和快乐的生活。过好这一生不仅仅是追求物质上的成功，更是追求内心的平静和满足。

过好这一生的关键在于找到自己的目标和使命。每个人都有自己独特的才能和激情，只有找到并实现这些才能和激情，才能真正过上有意义的生活。通过不断地探索和发展自己的潜力，人们可以找到自己的使命，并为之努力奋斗。

过好这一生还需要建立良好的人际关系。人是社会性动物，与他人的交往和沟通是人生中不可或缺的一部分。通过与家人、朋友和同事建立良好的关系，人们可以获得支持和帮助，同时也可以为他人带来快乐和帮助。良好的人际关系可以让人感到被理解和接纳，从而增强自信和幸福感。

过好这一生还需要保持身心健康。身体是革命的本钱，只有保持良好的身体健康，才能有足够的精力和动力去追求自己的目标和梦想。同时，心理健康也同样重要，只有保持积极的心态和情绪，才能更好地面对生活中的挑战和困难。

总的来说，过好这一生是一个综合性的目标，需要人们在各个方面都做出努力和改变。通过找到自己的使命、建立良好的人际关系、保持身心健康，人们可以过上充实、快乐和有意义的生活。希望每个人都能够找到属于自己的幸福之路，过好这一生。
